In [1]:
import anndata as ad
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import sys
import torch
import os
import matplotlib.pyplot as plt
import glob

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances

/usr/local/lib/python3.6/dist-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### silhouette score

In [22]:
meta = pd.read_csv('bmmc.meta.s3s4.csv')

ct_int = meta['cell_type'].astype('category').cat.codes.to_numpy().astype('int')

batch_int = meta['batch'].astype('category').cat.codes.to_numpy()


In [26]:
merge_list = glob.glob('results/raw/*embed.s3s4*') + glob.glob('results/raw/*umap.s3s4*')
merge_list[0]

'results/raw/bindSC_embed.s3s4.txt'

In [30]:
from sklearn.metrics import silhouette_score


method_id_list = []
for i in range(len(merge_list)):
    method_id = merge_list[i].split('/')[2].split('_')[0]
    method_id_list.append(method_id)

silh_score_df = pd.DataFrame({'method':method_id_list,
                              'cell_type_score':0,
                              'batch_score':0})

for i in range(len(merge_list)):
    merge = pd.read_csv(merge_list[i]).iloc[:,1:].to_numpy()
    silh_score_df.iloc[i,1] = silhouette_score(merge, ct_int)
    silh_score_df.iloc[i,2] = 1 - silhouette_score(merge, batch_int)


In [31]:
silh_score_df

,method,cell_type_score,batch_score
0,bindSC,0.111849,0.997807
1,GLUE,0.185618,0.992582
2,Liger,0.110812,0.982587
3,GLUE,0.172229,0.993985
4,GLUE,0.171599,0.993087
5,Liger,0.078424,0.980681
6,Liger,0.144200,0.975571
7,GLUE,0.170805,0.992963
8,Liger,0.067964,0.960554
9,GLUE,0.177198,0.992836


In [32]:
silh_score_df.to_csv('results/silh_score.s3s4.csv',index=False) #all

In [33]:
meta = pd.read_csv('bmmc.meta.s4s3.csv')

ct_int = meta['cell_type'].astype('category').cat.codes.to_numpy().astype('int')

batch_int = meta['batch'].astype('category').cat.codes.to_numpy()

merge_list = glob.glob('results/raw/*embed.s4s3*') + glob.glob('results/raw/*umap.s4s3*')
merge_list[0]

'results/raw/GLUE_embed.s4s3.3.txt'

In [35]:
from sklearn.metrics import silhouette_score


method_id_list = []
for i in range(len(merge_list)):
    method_id = merge_list[i].split('/')[2].split('_')[0]
    method_id_list.append(method_id)

silh_score_df = pd.DataFrame({'method':method_id_list,
                              'cell_type_score':0,
                              'batch_score':0})

for i in range(len(merge_list)):
    merge = pd.read_csv(merge_list[i]).iloc[:,1:].to_numpy()
    silh_score_df.iloc[i,1] = silhouette_score(merge, ct_int)
    silh_score_df.iloc[i,2] = 1 - silhouette_score(merge, batch_int)


In [36]:
silh_score_df

,method,cell_type_score,batch_score
0,GLUE,0.133556,0.992161
1,GLUE,0.145413,0.995229
2,GLUE,0.134867,0.995765
3,Liger,0.069232,0.980989
4,GLUE,0.144162,0.995813
5,Liger,0.064550,0.986489
6,bindSC,0.059268,0.997389
7,Liger,0.079965,0.987632
8,Liger,0.047466,0.990982
9,GLUE,0.137979,0.994160


In [37]:
silh_score_df.to_csv('results/silh_score.s4s3.csv',index=False) #all

### Label transfer accuracy - s3s4

In [2]:
file_list = glob.glob('results/raw/*acc.s3s4*')
file_list = sorted(file_list)
file_list[0]

'results/raw/GLUE_transfer_acc.s3s4.1.csv'

In [4]:
method_id_list = []

for i in range(len(file_list)):
    method_id = file_list[i].split('/')[2].split('_')[0]   
    method_id_list.append(method_id)

pred_df = pd.read_csv(file_list[0],index_col=0)

acc_ct_df = pd.DataFrame(np.zeros((len(file_list), pred_df.shape[0]+1)))

In [5]:
acc_ct_df.columns = ['method'] + pred_df['cell_type'].tolist()
acc_ct_df['method'] = method_id_list

In [6]:
for i in range(len(file_list)):
    if method_id_list[i] == 'Seurat':
        seurat = pd.read_csv(file_list[i])
        seurat = seurat.loc[seurat['Var1']==seurat['Var2'],:]
        seurat = seurat.iloc[:,2:]
        seurat.index = seurat['Var2'].to_numpy()
        seurat.columns = ['cell_type', 'Seurat']
        preded_ct = np.intersect1d(acc_ct_df.columns[1:],seurat.index)
        unpreded_ct = np.setdiff1d(acc_ct_df.columns[1:],seurat.index)
        seurat.insert
        for ct in unpreded_ct:
            seurat.loc[-1] = [ct, 0.0]
            seurat.index = seurat.index[0:-1].tolist() + [ct]
        seurat = seurat.loc[acc_ct_df.columns[1:],:]
        assert np.all(acc_ct_df.columns[1:].to_numpy() == seurat['cell_type'].to_numpy())
        acc_ct_df.iloc[i,1:] = seurat.iloc[:,1].to_numpy()
    else:
        pred_df = pd.read_csv(file_list[i],index_col=0)
        assert np.all(acc_ct_df.columns[1:].to_numpy() == pred_df['cell_type'].to_numpy())
        acc_ct_df.iloc[i,1:] = pred_df.iloc[:,1].to_numpy()

In [7]:
acc_ct_df

,method,B1 B,Naive CD20+ B,Erythroblast,CD4+ T activated,CD14+ Mono,Plasma cell,NK,CD8+ T,Proerythroblast,cDC2,MK/E prog,Transitional B,Normoblast,CD16+ Mono,CD4+ T naive,G/M prog,Lymph prog,pDC,HSC
0,GLUE,0.441417,0.818182,0.914352,0.235189,0.992400,0.000000,0.951244,0.839595,0.634812,0.257732,0.635359,0.618442,0.934066,0.581498,0.028249,0.567901,0.003831,0.657143,0.828125
1,GLUE,0.455041,0.859180,0.886574,0.328546,0.990228,0.000000,0.912438,0.841040,0.556314,0.463918,0.513812,0.763116,0.912088,0.563877,0.042373,0.691358,0.544061,0.814286,0.710938
2,GLUE,0.471390,0.848485,0.905093,0.202873,0.986971,0.000000,0.931343,0.825145,0.590444,0.298969,0.546961,0.742448,0.901099,0.511013,0.045198,0.617284,0.421456,0.800000,0.796875
3,GLUE,0.452316,0.786096,0.930556,0.308797,0.992400,0.000000,0.947264,0.838150,0.614334,0.216495,0.646409,0.600954,0.912088,0.669604,0.033898,0.604938,0.003831,0.650000,0.804688
4,GLUE,0.446866,0.787879,0.928241,0.276481,0.991314,0.000000,0.954229,0.806358,0.634812,0.288660,0.718232,0.575517,0.923077,0.656388,0.011299,0.586420,0.003831,0.785714,0.851562
5,GLUE,0.452316,0.864528,0.914352,0.330341,0.988056,0.000000,0.915423,0.838150,0.559727,0.278351,0.458564,0.791733,0.890110,0.541850,0.016949,0.709877,0.662835,0.771429,0.757812
6,GLUE,0.495913,0.786096,0.909722,0.319569,0.989142,0.000000,0.947264,0.812139,0.634812,0.319588,0.740331,0.578696,0.879121,0.638767,0.025424,0.598765,0.000000,0.657143,0.851562
7,GLUE,0.441417,0.798574,0.918981,0.305206,0.985885,0.000000,0.947264,0.822254,0.607509,0.371134,0.712707,0.602544,0.901099,0.563877,0.028249,0.679012,0.007663,0.800000,0.867188
8,Liger,0.188011,0.859180,0.655093,0.657092,0.940282,0.000000,0.960199,0.797688,0.552901,0.000000,0.552486,0.828299,0.813187,0.418502,0.070621,0.061728,0.555556,0.850000,0.906250
9,Liger,0.343324,0.713012,0.444444,0.290844,0.961998,0.000000,0.962189,0.702312,0.716724,0.000000,0.591160,0.834658,0.857143,0.493392,0.008475,0.006173,0.750958,0.814286,0.906250


In [8]:
file_list_2 = glob.glob('results/raw/*prediction.s3s4*')
file_list_2 = sorted(file_list_2)
file_list_2[0]

'results/raw/GLUE_prediction.s3s4.1.txt'

In [9]:
method_id_list = []
for i in range(len(file_list_2)):
    method_id = file_list_2[i].split('/')[2].split('_')[0]
    if 'UMAP' in file_list_2[i]:
        method_id = '%s (UMAP)'%(method_id)        
    method_id_list.append(method_id)


acc_df = pd.DataFrame({'method':method_id_list,
                              'total_acc':0})

for i in range(len(file_list_2)):
    pred_df = pd.read_csv(file_list_2[i])
    acc_df.iloc[i,1] = np.mean(pred_df['cell_type'] == pred_df['prediction'])




In [10]:
acc_ct_df.insert(1, 'total', acc_df['total_acc'].to_numpy())

In [11]:
acc_ct_df

,method,total,B1 B,Naive CD20+ B,Erythroblast,CD4+ T activated,CD14+ Mono,Plasma cell,NK,CD8+ T,...,cDC2,MK/E prog,Transitional B,Normoblast,CD16+ Mono,CD4+ T naive,G/M prog,Lymph prog,pDC,HSC
0,GLUE,0.667726,0.441417,0.818182,0.914352,0.235189,0.992400,0.000000,0.951244,0.839595,...,0.257732,0.635359,0.618442,0.934066,0.581498,0.028249,0.567901,0.003831,0.657143,0.828125
1,GLUE,0.703801,0.455041,0.859180,0.886574,0.328546,0.990228,0.000000,0.912438,0.841040,...,0.463918,0.513812,0.763116,0.912088,0.563877,0.042373,0.691358,0.544061,0.814286,0.710938
2,GLUE,0.687630,0.471390,0.848485,0.905093,0.202873,0.986971,0.000000,0.931343,0.825145,...,0.298969,0.546961,0.742448,0.901099,0.511013,0.045198,0.617284,0.421456,0.800000,0.796875
3,GLUE,0.672149,0.452316,0.786096,0.930556,0.308797,0.992400,0.000000,0.947264,0.838150,...,0.216495,0.646409,0.600954,0.912088,0.669604,0.033898,0.604938,0.003831,0.650000,0.804688
4,GLUE,0.670214,0.446866,0.787879,0.928241,0.276481,0.991314,0.000000,0.954229,0.806358,...,0.288660,0.718232,0.575517,0.923077,0.656388,0.011299,0.586420,0.003831,0.785714,0.851562
5,GLUE,0.706980,0.452316,0.864528,0.914352,0.330341,0.988056,0.000000,0.915423,0.838150,...,0.278351,0.458564,0.791733,0.890110,0.541850,0.016949,0.709877,0.662835,0.771429,0.757812
6,GLUE,0.672564,0.495913,0.786096,0.909722,0.319569,0.989142,0.000000,0.947264,0.812139,...,0.319588,0.740331,0.578696,0.879121,0.638767,0.025424,0.598765,0.000000,0.657143,0.851562
7,GLUE,0.674914,0.441417,0.798574,0.918981,0.305206,0.985885,0.000000,0.947264,0.822254,...,0.371134,0.712707,0.602544,0.901099,0.563877,0.028249,0.679012,0.007663,0.800000,0.867188
8,Liger,0.684174,0.188011,0.859180,0.655093,0.657092,0.940282,0.000000,0.960199,0.797688,...,0.000000,0.552486,0.828299,0.813187,0.418502,0.070621,0.061728,0.555556,0.850000,0.906250
9,Liger,0.646994,0.343324,0.713012,0.444444,0.290844,0.961998,0.000000,0.962189,0.702312,...,0.000000,0.591160,0.834658,0.857143,0.493392,0.008475,0.006173,0.750958,0.814286,0.906250


In [12]:
acc_ct_df.to_csv('results/label_trasfer_acc.s3s4.csv',index=False)

### Label transfer accuracy - s4s3

In [13]:
file_list = glob.glob('results/raw/*acc.s4s3*')
file_list = sorted(file_list)
file_list[0]

'results/raw/GLUE_transfer_acc.s4s3.1.csv'

In [14]:
method_id_list = []

for i in range(len(file_list)):
    method_id = file_list[i].split('/')[2].split('_')[0]   
    method_id_list.append(method_id)

pred_df = pd.read_csv(file_list[0],index_col=0)

acc_ct_df = pd.DataFrame(np.zeros((len(file_list), pred_df.shape[0]+1)))

In [15]:
acc_ct_df.columns = ['method'] + pred_df['cell_type'].tolist()
acc_ct_df['method'] = method_id_list

In [16]:
for i in range(len(file_list)):
    if method_id_list[i] == 'Seurat':
        seurat = pd.read_csv(file_list[i])
        seurat = seurat.loc[seurat['Var1']==seurat['Var2'],:]
        seurat = seurat.iloc[:,2:]
        seurat.index = seurat['Var2'].to_numpy()
        seurat.columns = ['cell_type', 'Seurat']
        preded_ct = np.intersect1d(acc_ct_df.columns[1:],seurat.index)
        unpreded_ct = np.setdiff1d(acc_ct_df.columns[1:],seurat.index)
        seurat.insert
        for ct in unpreded_ct:
            seurat.loc[-1] = [ct, 0.0]
            seurat.index = seurat.index[0:-1].tolist() + [ct]
        seurat = seurat.loc[acc_ct_df.columns[1:],:]
        assert np.all(acc_ct_df.columns[1:].to_numpy() == seurat['cell_type'].to_numpy())
        acc_ct_df.iloc[i,1:] = seurat.iloc[:,1].to_numpy()
    else:
        pred_df = pd.read_csv(file_list[i],index_col=0)
        assert np.all(acc_ct_df.columns[1:].to_numpy() == pred_df['cell_type'].to_numpy())
        acc_ct_df.iloc[i,1:] = pred_df.iloc[:,1].to_numpy()

In [17]:
file_list_2 = glob.glob('results/raw/*prediction.s4s3*')
file_list_2 = sorted(file_list_2)
file_list_2[0]

'results/raw/GLUE_prediction.s4s3.1.txt'

In [18]:
method_id_list = []
for i in range(len(file_list_2)):
    method_id = file_list_2[i].split('/')[2].split('_')[0]
    if 'UMAP' in file_list_2[i]:
        method_id = '%s (UMAP)'%(method_id)        
    method_id_list.append(method_id)


acc_df = pd.DataFrame({'method':method_id_list,
                              'total_acc':0})

for i in range(len(file_list_2)):
    pred_df = pd.read_csv(file_list_2[i])
    acc_df.iloc[i,1] = np.mean(pred_df['cell_type'] == pred_df['prediction'])




In [19]:
acc_ct_df.insert(1, 'total', acc_df['total_acc'].to_numpy())

In [20]:
acc_ct_df

,method,total,NK,Normoblast,Transitional B,Lymph prog,Naive CD20+ B,Erythroblast,HSC,CD14+ Mono,...,pDC,B1 B,CD8+ T,CD4+ T activated,MK/E prog,CD16+ Mono,CD4+ T naive,Proerythroblast,ILC,Plasma cell
0,GLUE,0.695602,0.953722,0.529412,0.831776,0.524038,0.773333,0.719101,0.613281,0.930345,...,0.377309,0.894737,0.891192,0.374332,0.841121,0.849558,0.304348,0.958763,0.0,0.181818
1,GLUE,0.682310,0.943662,0.529412,0.816199,0.225962,0.829333,0.657303,0.695312,0.945517,...,0.348285,0.833333,0.854922,0.491979,0.878505,0.818584,0.206522,0.896907,0.0,0.727273
2,GLUE,0.688874,0.931590,0.529412,0.825545,0.370192,0.829333,0.716292,0.675781,0.953793,...,0.313984,0.780702,0.917098,0.229947,0.906542,0.796460,0.195652,0.979381,0.0,0.818182
3,GLUE,0.681162,0.965795,0.537815,0.937695,0.461538,0.973333,0.696629,0.660156,0.922759,...,0.166227,0.078947,0.891192,0.411765,0.915888,0.805310,0.293478,0.969072,0.0,0.090909
4,GLUE,0.695766,0.961771,0.613445,0.875389,0.500000,0.778667,0.646067,0.644531,0.924828,...,0.377309,0.912281,0.857513,0.556150,0.850467,0.827434,0.282609,0.938144,0.0,0.272727
5,GLUE,0.685264,0.953722,0.521008,0.769470,0.480769,0.826667,0.702247,0.679688,0.936552,...,0.253298,0.921053,0.896373,0.347594,0.915888,0.862832,0.184783,0.855670,0.0,0.454545
6,GLUE,0.658188,0.955734,0.487395,0.838006,0.298077,0.920000,0.640449,0.550781,0.931034,...,0.284960,0.307018,0.844560,0.417112,0.859813,0.805310,0.184783,0.896907,0.0,0.363636
7,GLUE,0.675911,0.967807,0.529412,0.757009,0.403846,0.746667,0.730337,0.652344,0.933793,...,0.263852,0.885965,0.852332,0.422460,0.878505,0.823009,0.239130,0.958763,0.0,0.272727
8,Liger,0.453069,0.796781,0.521008,0.647975,0.274038,0.688000,0.516854,0.835938,0.404828,...,0.738786,0.421053,0.432642,0.663102,0.130841,0.022124,0.184783,0.505155,0.0,0.000000
9,Liger,0.480309,0.863179,0.571429,0.725857,0.302885,0.626667,0.191011,0.921875,0.460690,...,0.788918,0.438596,0.455959,0.652406,0.168224,0.513274,0.184783,0.360825,0.0,0.181818


In [21]:
acc_ct_df.to_csv('results/label_trasfer_acc.s4s3.csv',index=False)